In [ ]:
import os
import re

import numpy as np

import biobeam
import gputools
import pyopencl
from matplotlib import pyplot as plt
from skimage.io import imsave
from skimage.io import imread_collection

from run_sim import sim_lightsheet, split_str_digit


%matplotlib inline
plt.rcParams['figure.figsize'] = [12,12]

In [ ]:
def simple_downsample(a, factors):
    """
    very simple subsampling of array
    
    Parameters
    ----------
    a: array
    the array to downsample
    factors: int or sequence of ints
    downsampling factors
    
    Returns
    -------
    a2: array
    subsampled version of a
    """
    
    if np.isscalar(factors):
        factors = [factors] * len(a.shape)
    
    a2 = a
    for i,f in enumerate(factors):
        a2 = np.take(a2, np.arange(0, a2.shape[i], f),axis=i)
        
    return a2

In [ ]:
# set and check which gpu is used
gputools.init_device(id_platform = 0, id_device = 1)
gputools.get_device()

In [ ]:
# call sim_lightsheet

if __name__ == '__main__':
    
    base_dir = '/Volumes/davidh-ssd/mv-sim/sim_tissue/'
    
    params = [
        (False, 200, 400),
    ]
    ''' (False, 400, 400),
        (False, 550, 400),
        (True, 250, 400),
        (True, 400, 400),
        (True, 550, 400),
        (False, 250, 600),
        (False, 400, 600),
        (False, 550, 600),
        (True, 250, 600),
        (True, 400, 600),
        (True, 550, 600)
    ]
    '''
    for (right_illum, ls_pos, lam) in params:
        out_dir = 'sim-biobeam2-illum_{}-ls_pos_{}-lam-{}'.format('right' if right_illum else 'left', ls_pos, lam)
        sim_lightsheet(base_dir, out_dir, right_illum=right_illum, ls_pos=ls_pos, lam=lam, planes_to_simulate=[128])
    print('ALL DONE.')
    

In [ ]:
# illumination from right side
right_illum = True
# phantom size in microns
physical_dims = (400,400,50)
# LS position
ls_pos = 200

In [ ]:
# load tiff series test data

sequence_pattern = 'sim(\d+).tif'
in_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-phantom/'

files = os.listdir(in_path)
files.sort(key=lambda f: split_str_digit(f))
files

ic = imread_collection([os.path.join(in_path, f) for f in files])
img = ic.concatenate()

img = np.pad(img, ((16,16),(0,0),(0,0)), "constant")
if right_illum:
    img = np.flip(img, 2)

In [ ]:
signal = img
dn = (signal / np.max(signal)) * 0.01

#create a microscope simulator
m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal, zfoc_illum=physical_dims[0]/2 - ls_pos,
                       NA_illum= .1, NA_detect=.45,
                       n_volumes=8,
                       size = physical_dims, n0 = 1.33)

In [ ]:
out_fmt = 'sim-bbeam-{}-{}.tif'
out_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-right2/'

if not os.path.exists(out_path):
    os.makedirs(out_path)

out = np.zeros((img.shape[0] - 16*2,) + img.shape[1:], dtype=img.dtype)

for i in range(out.shape[0]):
    
    if i != 155:
        continue
    
    cz = i - (img.shape[0] - 16*2) // 2
    cz = cz * m._bpm_detect.units[-1]

    image = m.simulate_image_z(cz=cz, psf_grid_dim=(16,16), conv_sub_blocks=(8,8))
    out[i] = image[16] if not right_illum else np.flip(image[16], 2)
    
    # save immediately
    imsave(os.path.join(out_path, out_fmt.format("left" if not right_illum else "right", i)), out[i])
    
    print("{} of {} done".format(i+1, out.shape[0]))

In [ ]:
image[16] if not right_illum else np.flip(image[16], 1).shape
#out.shape
np.array(img.shape) - 32

from scipy.ndimage import zoom

zoom(img, [1,2,2]).shape

In [ ]:


simple_downsample(img, [1,2,2]).shape

In [ ]:
_ = """

pl = pyopencl.get_platforms()[0]
pl.get_devices()
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

gputools.init_device(id_platform = 0, id_device = 1)

signal = read_tiff_stack('/Users/david/Desktop/Image 0-1.tif', False)
signal = np.swapaxes(signal, 0,2)
dn = (signal > 0) * 0.01

#create a microscope simulator
m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal,
                       NA_illum= .1, NA_detect=.45,
                       n_volumes=1,
                       size = (128,512,512), n0 = 1.33)

image = m.simulate_image_z(cz=0, psf_grid_dim=(16,16), conv_sub_blocks=(2,2))


print(image.shape)
plt.imshow(image[16])
"""